# GHZ Distillation Protocol

In [1]:
!pip install numpy matplotlib qulacs

     ------------------------------------- 619.5/619.5 kB 19.7 MB/s eta 0:00:00


In [2]:
from qulacs import QuantumCircuit, QuantumState, DensityMatrix
from qulacs.state import make_superposition, make_mixture, make_superposition, tensor_product, partial_trace
from qulacs.gate import H, X, CNOT, Z, Measurement, Identity
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def print_state_vector(vec: np.ndarray, eps: float = 1e-10) -> None:
    n = int(np.log2(len(vec) + 1e-10))
    for ind in range(2**n):
        if np.abs(vec[ind]) < eps:
            continue
        s = bin(ind)[2:].zfill(n)
        print(f"{vec[ind]:.3f} |{s}>")

def print_density_matrix(dm: np.ndarray, eps: float = 1e-10) -> None:
    ee, ev = np.linalg.eigh(dm)
    for ei, eval in enumerate(ee):
        if eval < eps:
            continue
        print(f"prob = {eval}")
        print_state_vector(ev[:, ei])

## Creating a desiable GHZ-state
$\begin{equation}|\Psi\rangle = \frac{|1010\rangle + |0101\rangle}{2}\end{equation}$  
`ghz_state` is Qulacs's representation of density matrix $|\Psi\rangle\langle\Psi|$

In [4]:
ghz_state = DensityMatrix(4)
state1010 = QuantumState(4)
state0101 = QuantumState(4)
state1010.set_computational_basis(0b1010)
state0101.set_computational_basis(0b0101)
ghz_vector = make_superposition(1, state1010, 1, state0101)
ghz_state = make_mixture(1.0, ghz_vector, 0, ghz_vector)
ghz_state.normalize(ghz_state.get_squared_norm())

## Applying depolarinzing noise to `ghz_state`
$\begin{equation}
\rho_{\text{imperfect}} = (1 - p_n) |\Psi\rangle\langle\Psi| + \frac{p_n}{16} I
\end{equation}$

In [ ]:
def create_rho_imperfect(p_n):
    mat = (1 - p_n) * ghz_state.get_matrix() + p_n / 16 * np.eye(16)
    state = DensityMatrix(4)
    state.load(mat)
    return state

## Applying photon loss to create `rho_raw`
$\begin{align}
\rho_{\text{raw}} &= \alpha \rho_{\text{imperfect}} + \sum_{i=1}^{5} \beta_i |\phi_i\rangle\langle\phi_i| \nonumber \\
    &= \mathcal{N} \left[p_0Y_0^2|\Psi\rangle\langle\Psi| + p_1Y_1^2 \sum_{i=1}^4
    |\phi_i\rangle\langle\phi_i| + 2 p_2Y_2^2 |\phi_5\rangle\langle\phi_5| \right] \nonumber
\end{align}$

where \\
$\begin{align}\{|\phi_i\rangle\} &= \{|1110\rangle, |1101\rangle, |1011\rangle, |0111\rangle, |1111\rangle\} \nonumber \\
p_mY_m^2 &= \frac{1}{4}\eta^2(1-\eta)^m a^{4-2m}b^{4+2m} \nonumber
\end{align}$

`p_emi`: photon emittion rate  
`p_loss`: photon loss rate

In [ ]:
def create_rho_raw(rho_imperfect, p_emi, p_loss):
    a = np.sqrt(1 - p_emi)
    b = np.sqrt(p_emi)
    eta = 1 - p_loss
    pY2 = [(eta ** 2) * ((1 - eta) ** m) / 4 * (a ** (4 - 2*m)) * (b ** (4 + 2*m)) for m in [0, 1, 2]]
    state1110 = DensityMatrix(4)
    state1110.set_computational_basis(0b1110)
    state1101 = DensityMatrix(4)
    state1101.set_computational_basis(0b1101)
    state1011 = DensityMatrix(4)
    state1011.set_computational_basis(0b1011)
    state0111 = DensityMatrix(4)
    state0111.set_computational_basis(0b0111)
    state1111 = DensityMatrix(4)
    tmp1 = make_mixture(1,
        make_mixture(1,
            make_mixture(1, state1110, 1, state1101),
            1, state1011),
        1, state0111)
    state1111.set_computational_basis(0b1111)
    tmp2 = make_mixture(1, state1111, 1, state1111)
    rho_X = make_mixture(pY2[0], rho_imperfect, 1.0, make_mixture(pY2[1], tmp1, pY2[2], tmp2))
    rho_X.normalize(rho_X.get_squared_norm())
    return rho_X

## Distillation protocol

- raw protocol: Does nothing
- basic protocol
- medium protocol

In [ ]:
class Protocol:
    def __init__(self, p_n, p_emi, p_loss):
        self.p_n = p_n
        self.p_emi = p_emi
        self.p_loss = p_loss
        self.rho_imperfect = create_rho_imperfect(p_n)
        self.rho_raw = create_rho_raw(self.rho_imperfect, p_emi, p_loss)
        self.basic_prob = None
        self.medium_prob = None

    def apply_raw_protocol(self):
        return self.rho_raw

    def apply_basic_protocol(self):
        circuit = QuantumCircuit(8)
        circuit.add_CNOT_gate(0, 4)
        circuit.add_CNOT_gate(1, 5)
        circuit.add_CNOT_gate(2, 6)
        circuit.add_CNOT_gate(3, 7)
        circuit.add_P1_gate(4)
        circuit.add_P1_gate(5)
        circuit.add_P1_gate(6)
        circuit.add_P1_gate(7)
        rho_raw = self.rho_raw.copy()
        rho_basic = tensor_product(rho_raw, rho_raw)
        circuit.update_quantum_state(rho_basic)
        rho_basic_traced = partial_trace(rho_basic, [4, 5, 6, 7])
        self.basic_prob = rho_basic_traced.get_squared_norm()
        rho_basic_traced.normalize(rho_basic_traced.get_squared_norm())
        return rho_basic_traced

    def apply_medium_protocol(self):
        circuit = QuantumCircuit(8)
        circuit.add_CZ_gate(0, 4)
        circuit.add_CZ_gate(1, 5)
        circuit.add_CZ_gate(2, 6)
        circuit.add_CZ_gate(3, 7)
        circuit.add_H_gate(4)
        circuit.add_P1_gate(4)
        circuit.add_H_gate(5)
        circuit.add_P1_gate(5)
        circuit.add_H_gate(6)
        circuit.add_P1_gate(6)
        circuit.add_H_gate(7)
        circuit.add_P1_gate(7)
        rho_basic = self.apply_basic_protocol().copy()
        rho_medium = tensor_product(rho_basic, rho_basic)
        circuit.update_quantum_state(rho_medium)
        rho_medium_traced = partial_trace(rho_medium, [4, 5, 6, 7])
        self.medium_prob = rho_medium_traced.get_squared_norm()
        rho_medium_traced.normalize(rho_medium_traced.get_squared_norm())
        return rho_medium_traced

    def success_rate(self):
        eta = 1 - self.p_loss
        return 0.5 * eta ** 2 * self.p_emi ** 2 * (1 - self.p_emi * eta) ** 2

In [ ]:
def fidelity(rho1, rho2):
    return np.abs(np.trace(np.dot(rho1.get_matrix(), rho2.get_matrix())))

# Simulation
## Parameters
- `p_n` = 0.0106
- `p_emi` = 0.04
- `p_loss` = 1 - η
    - near term: 1 - 0.0046 = 0.9954
    - future term: 1 - 0.4472 = 0.5528

In [ ]:
protocol = Protocol(p_n = 0.0106, p_emi = 0.04, p_loss = 0.9954)
#protocol = Protocol(p_n = 0.0106, p_emi = 0.04, p_loss = 0.5528)

In [ ]:
raw = protocol.apply_raw_protocol()
basic = protocol.apply_basic_protocol()
medium = protocol.apply_medium_protocol()
print(fidelity(ghz_state, raw))
print(fidelity(ghz_state, basic))
print(fidelity(ghz_state, medium))

0.8466846277404558
0.9981946238150656
0.99842097200263


In [ ]:
raw.get_matrix()

array([[0.00056656+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.00056656+0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.00056656+0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.00056656+0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.   

In [ ]:
#mat = raw.get_matrix().real
#np.savetxt('data/ghz_raw.csv', mat, delimiter=',')

## Success rate of raw state

In [ ]:
protocol.success_rate()

1.6921771069114723e-08

## Success rate of distillation protocol

In [ ]:
print(protocol.basic_prob)
print(protocol.medium_prob * protocol.basic_prob)

0.3590858750074868
0.0447948835544981
